In [17]:
import os
import pandas as pd
from IPython.display import display, HTML
import re

#path = "c:\\data\\"
path = "/Users/jeff/temp/dissertation"


filename_read = os.path.join(path,"workload.csv")
df = pd.read_csv(filename_read)


# Experiment 1

In [19]:
lookup = {'diff-y0' : 'Difference',
'log-y0' : 'Log',
'poly-y0' : 'Polynomial',
'poly2-y0' : 'Polynomial2',
'pow-y0' : 'Power',
'ratio-y0' : 'Ratio',
'ratio_diff-y0' : 'Ratio Difference',
'ratio_poly-y0' : 'Ratio Polynomial',
'ratio_poly2-y0' : 'Ratio Polynomial2',
'sqrt-y0' : 'Square Root'}


ex1 = df[df.name == 'experiment-1'][['dataset','algorithm','predictors','cycle','result','result-raw']]
f = {'result':['min','mean','std']}
ex1_summary = ex1.groupby(['algorithm']).agg(f)

ex1_summary.columns = [x[1] for x in ex1_summary.columns.values]
ex1_summary.insert(0,'algorithm', ex1_summary.index)
ex1_summary.insert(1, 'name' ,[re.search('\:(.*)\|', x).group(1) for x in ex1_summary.algorithm])
ex1_summary.insert(0, 'set' , [('gp' if 'gp' in x else 'neural') for x in ex1_summary.algorithm])
ex1_summary = ex1_summary[ex1_summary.name.isin(lookup.keys())]
ex1_summary.insert(1, 'expression' ,[lookup[x] for x in ex1_summary.name])
ex1_summary.sort_values(by=['expression'],inplace=True)
ex1_summary.index = range(len(ex1_summary))
ex1_summary.insert(0, 'num', ["{}-{}".format(1,x+1) for x in range(len(ex1_summary))])



In [20]:
# Experiment 1 neural results
ex1_summary_neural = ex1_summary[ex1_summary['set'] == 'neural'][['num','expression','min','mean','std']]
display(ex1_summary_neural)
ex1_summary_neural.to_csv(os.path.join(path,"diss_ex1_neural.csv"),index=False)

,num,expression,min,mean,std
1,1-2,Difference,0.011845,0.653509,0.358701
3,1-4,Log,0.200801,0.543885,0.242550
5,1-6,Polynomial,3.500087,3.500112,0.000023
7,1-8,Polynomial2,1.137168,1.137300,0.000278
9,1-10,Power,0.239435,0.275668,0.033076
11,1-12,Ratio,27.448782,27.467884,0.013711
13,1-14,Ratio Difference,22.977635,22.982264,0.003387
14,1-15,Ratio Polynomial,10.229576,10.249734,0.011422
16,1-17,Ratio Polynomial2,0.058682,0.101589,0.053896
19,1-20,Square Root,0.014715,0.106929,0.115772


In [21]:
# Experiment 1 GP results
ex1_summary_gp = ex1_summary[ex1_summary['set'] == 'gp'][['num','expression','min','mean','std']]
display(ex1_summary_gp)
ex1_summary_gp.to_csv(os.path.join(path,"diss_ex1_gp.csv"),index=False)

,num,expression,min,mean,std
0,1-1,Difference,0.000000,0.114616,0.256290
2,1-3,Log,0.071247,0.219587,0.136508
4,1-5,Polynomial,0.936565,1.250593,0.363454
6,1-7,Polynomial2,0.304300,0.710227,0.303976
8,1-9,Power,0.000000,0.000000,0.000000
10,1-11,Ratio,0.000000,0.988118,2.149678
12,1-13,Ratio Difference,22.968550,22.977673,0.007437
15,1-16,Ratio Polynomial,7.806944,8.068410,0.563785
17,1-18,Ratio Polynomial2,0.097735,0.181037,0.046576
18,1-19,Square Root,0.000028,0.046927,0.037877


# Experiment 2

In [29]:
ex2 = df[df.name == 'experiment-2'][['dataset','algorithm','predictors','cycle','result','result-raw']]
#ex2 = ex2[ex2.dataset != 'feature_eng.csv']

f = {'result-raw':['min','mean','std']}
ex2_summary = ex2.groupby(['algorithm','dataset']).agg(f)

ex2_summary.columns = [x[1] for x in ex2_summary.columns.values]
ex2_summary.insert(0,'algorithm', [x[0] for x in ex2_summary.index])
ex2_summary.insert(0,'dataset', [x[1] for x in ex2_summary.index])
ex2_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex2_summary.dataset])
ex2_summary.insert(0, 'set' , [('gp' if 'gp' in x else 'neural') for x in ex2_summary.algorithm])
ex2_summary.index = range(len(ex2_summary))
ex2_summary.sort_values(by=['set','name'],inplace=True)

ex2_summary_neural = ex2_summary[ex2_summary['set'] == 'neural'][['name','min','mean','std']]
ex2_summary_neural.columns = ['name','neural_min','neural_mean','neural_std']
ex2_summary_gp = ex2_summary[ex2_summary['set'] == 'gp'][['name','min','mean','std']]
ex2_summary_gp.columns = ['name','gp_min','gp_mean','gp_std']
ex2_summary = pd.merge(ex2_summary_neural,ex2_summary_gp,how="left",on=['name'])

ex2_summary.insert(0, 'num', ["{}-{}".format(2,x+1) for x in range(len(ex2_summary))])

#display(ex2_summary_neural)
#display(ex2_summary_gp)
display(ex2_summary)
ex2_summary_neural.to_csv(os.path.join(path,"diss_ex2.csv"),index=False)

,num,name,neural_min,neural_mean,neural_std,gp_min,gp_mean,gp_std
0,2-1,abalone,2.157315,2.600538,0.553925,2.290352,2.515296,0.173115
1,2-2,auto-mpg,2.234662,3.403514,2.326207,4.764571,5.829995,0.917775
2,2-3,bupa,0.429799,0.442881,0.011279,0.463727,0.486231,0.014515
3,2-4,covtype,0.377724,0.408585,0.022559,NaN,NaN,NaN
4,2-5,crx,0.321929,0.331049,0.008736,0.391675,0.391675,0.000000
5,2-6,forestfires,0.061136,0.062294,0.000966,NaN,NaN,NaN
6,2-7,glass,0.319512,0.339805,0.015664,NaN,NaN,NaN
7,2-8,heart_cleveland,0.316120,0.339291,0.019778,NaN,NaN,NaN
8,2-9,hepatitis,0.434181,0.450453,0.017165,0.426121,0.452999,0.026263
9,2-10,horse-colic,0.344475,0.357490,0.010901,NaN,NaN,NaN


# Experiment 3

In [30]:
ex3 = df[df.name == 'experiment-3'][['dataset','algorithm','predictors','cycle','result','result-raw']]

f = {'result-raw':['min','mean','std']}
ex3_summary = ex3.groupby(['dataset']).agg(f)

ex3_summary.columns = [x[1] for x in ex3_summary.columns.values]
ex3_summary.insert(0,'dataset', ex3_summary.index)
ex3_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex3_summary.dataset])
ex3_summary.index = range(len(ex3_summary))
ex3_summary.drop('dataset', axis=1, inplace=True)
ex3_summary.columns = ['name','ensemble_min','ensemble_mean','ensemble_std']

ex3_summary = pd.merge(ex3_summary,ex2_summary_neural,how="left",on=['name'])
ex3_summary = pd.merge(ex3_summary,ex2_summary_gp,how="left",on=['name'])
ex3_summary.drop(['neural_std','neural_min','gp_std','gp_min'], axis=1, inplace=True)
ex3_summary.insert(0, 'num', ["{}-{}".format(3,x+1) for x in range(len(ex3_summary))])

display(ex3_summary)
ex3_summary.to_csv(os.path.join(path,"diss_ex3.csv"),index=False)

,num,name,ensemble_min,ensemble_mean,ensemble_std,neural_mean,gp_mean
0,3-1,abalone,2.103613,2.314365,0.300377,2.600538,2.515296
1,3-2,auto-mpg,2.249562,4.416206,2.884154,3.403514,5.829995
2,3-3,bupa,0.409062,0.450952,0.030612,0.442881,0.486231
3,3-4,crx,0.319514,0.338281,0.015350,0.331049,0.391675
4,3-5,hepatitis,0.447426,0.453949,0.006114,0.450453,0.452999
5,3-6,pima-indians-diabetes,0.387386,0.427292,0.032828,0.417629,0.455608
6,3-7,wcbreast_wdbc,0.084456,0.105642,0.020242,0.087830,0.302574
7,3-8,wcbreast_wpbc,0.343728,0.390409,0.033343,0.380398,0.431375


# Experiment 6

In [35]:
ex6 = df[df.name == 'experiment-6'][['dataset','algorithm','predictors','cycle','result','result-raw']]

f = {'result-raw':['min','mean','std']}
ex6_summary = ex6.groupby(['algorithm','dataset']).agg(f)

ex6_summary.columns = [x[1] for x in ex6_summary.columns.values]
ex6_summary.insert(0,'algorithm', [x[0] for x in ex6_summary.index])
ex6_summary.insert(0,'dataset', [x[1] for x in ex6_summary.index])
ex6_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex6_summary.dataset])
ex6_summary.index = range(len(ex6_summary))
ex6_summary.drop(['dataset','algorithm'], axis=1, inplace=True)
ex6_summary.columns = ['name','auto_min','auto_mean','auto_std']
ex6_summary = pd.merge(ex6_summary,ex3_summary,how="left",on=['name'])
ex6_summary = pd.merge(ex6_summary,ex2_summary_neural,how="left",on=['name'])
ex6_summary.neural_mean_x.fillna(ex6_summary.neural_mean_y,inplace=True)
ex6_summary.drop(['auto_min','auto_std','ensemble_min','ensemble_std','neural_mean_y','neural_min','neural_std','num'], 
                 axis=1, inplace=True)
ex6_summary.rename(columns={'neural_mean_x':'neural_mean'},inplace=True)
ex6_summary['best_non_fe'] = ex6_summary[['ensemble_mean','neural_mean','gp_mean']].min(axis=1,skipna=True)
ex6_summary['diff'] = ex6_summary.auto_mean - ex6_summary.best_non_fe
ex6_summary['pct'] = (ex6_summary.auto_mean - ex6_summary.best_non_fe) / ex6_summary.best_non_fe

ex6_summary.sort_values(by=['name'],inplace=True)
ex6_summary.index = range(len(ex6_summary))
ex6_summary.insert(0, 'num', ["{}-{}".format(6,x+1) for x in range(len(ex6_summary))])
ex6_summary = ex6_summary[['num','name','ensemble_mean','neural_mean','gp_mean','best_non_fe','auto_mean','diff','pct']]


display(ex6_summary)
ex6_summary.to_csv(os.path.join(path,"diss_ex6.csv"),index=False)

,num,name,ensemble_mean,neural_mean,gp_mean,best_non_fe,auto_mean,diff,pct
0,6-1,abalone,2.314365,2.600538,2.515296,2.314365,2.902405,0.588040,0.254083
1,6-2,auto-mpg,4.416206,3.403514,5.829995,3.403514,4.600421,1.196908,0.351668
2,6-3,bupa,0.450952,0.442881,0.486231,0.442881,0.463933,0.021051,0.047532
3,6-4,covtype,NaN,0.408585,NaN,0.408585,0.407572,-0.001014,-0.002481
4,6-5,crx,0.338281,0.331049,0.391675,0.331049,0.323032,-0.008017,-0.024217
5,6-6,forestfires,NaN,0.062294,NaN,0.062294,0.061944,-0.000351,-0.005627
6,6-7,glass,NaN,0.339805,NaN,0.339805,0.318118,-0.021687,-0.063821
7,6-8,heart_cleveland,NaN,0.339291,NaN,0.339291,0.318942,-0.020349,-0.059976
8,6-9,hepatitis,0.453949,0.450453,0.452999,0.450453,0.447182,-0.003271,-0.007262
9,6-10,horse-colic,NaN,0.357490,NaN,0.357490,0.381531,0.024041,0.067250


In [90]:
from scipy.stats import ttest_ind

improve_datasets = ex6_summary[ex6_summary.pct<0].name.tolist()

for dataset in improve_datasets:
    filename = dataset + ".csv"
    a = df[ (df.dataset == filename) & (df.name=='experiment-2') & (df.algorithm.str.contains('neural'))]['result-raw'].tolist()
    b = df[ (df.dataset == filename) & (df.name=='experiment-6')]['result-raw'].tolist()
    t = ttest_ind(a, b)
    print("{}:{}".format(dataset,t))
    

covtype:Ttest_indResult(statistic=0.10046231009051176, pvalue=0.9224493732825525)
crx:Ttest_indResult(statistic=1.2769019840423801, pvalue=0.23745029212413638)
forestfires:Ttest_indResult(statistic=0.40409837729768699, pvalue=0.69672769230840004)
glass:Ttest_indResult(statistic=1.953988885343269, pvalue=0.086457632649443283)
heart_cleveland:Ttest_indResult(statistic=1.9539388522999459, pvalue=0.086464317953459713)
hepatitis:Ttest_indResult(statistic=0.42613130087859524, pvalue=0.68124599786361473)
iris:Ttest_indResult(statistic=0.31626554925468348, pvalue=0.75989533575105361)
pima-indians-diabetes:Ttest_indResult(statistic=0.63048100260782314, pvalue=0.54596418941503799)
wine:Ttest_indResult(statistic=0.38772689301129704, pvalue=0.7083305622960534)


In [91]:
import numpy as np

a = df[ (df.dataset == 'wine.csv') & (df.name=='experiment-2') & (df.algorithm.str.contains('neural'))]['result-raw'].tolist()
b = df[ (df.dataset == 'wine.csv') & (df.name=='experiment-6')]['result-raw'].tolist()
print(a)
print(b)
print(np.mean(a))
print(np.mean(b))
ttest_ind(a, b)






[0.12598719954675955, 0.13553334024017952, 0.18184824186332699, 0.20281457524178892, 0.048077500439004203]
[0.1880325154548978, 0.032684151744574563, 0.032684151744574563, 0.16417290870679765, 0.1889822365046136]
0.138852171466
0.121311192831


Ttest_indResult(statistic=0.38772689301129704, pvalue=0.7083305622960534)

In [77]:
f = {'result-raw':['min','mean','std']}
ex2.groupby(['algorithm','dataset']).agg(f)

result-raw            \
                                                           min      mean   
algorithm                 dataset                                          
gp-r:a16|nrmse            crx.csv                     0.391675  0.391675   
gp-r:class|nrmse          hepatitis.csv               0.426121  0.452999   
                          pima-indians-diabetes.csv   0.447533  0.455608   
gp-r:diagnosis|nrmse      wcbreast_wdbc.csv           0.232923  0.302574   
gp-r:mpg|nrmse            auto-mpg.csv                4.764571  5.829995   
gp-r:outcome|nrmse        wcbreast_wpbc.csv           0.415125  0.431375   
gp-r:rings|nrmse          abalone.csv                 2.290352  2.515296   
gp-r:selector|nrmse       bupa.csv                    0.463727  0.486231   
neural-c:area|nrmse       forestfires.csv             0.061136  0.062294   
neural-c:class|nrmse      hepatitis.csv               0.434181  0.450453   
                          pima-indians-diabetes.csv   0.390089  0.417629   
                          wine.csv                    0.048078  0.138852   
neural-c:cover_type|nrmse covtype.csv                 0.377724  0.408585   
neural-c:crim|nrmse       housing.csv                 0.044555  0.044607   
neural-c:diagnosis|nrmse  wcbreast_wdbc.csv           0.080323  0.087830   
neural-c:num|nrmse        heart_cleveland.csv         0.316120  0.339291   
neural-c:outcome|nrmse    horse-colic.csv             0.344475  0.357490   
                          wcbreast_wpbc.csv           0.322736  0.380398   
neural-c:species|nrmse    iris.csv                    0.097372  0.290452   
neural-c:type|nrmse       glass.csv                   0.319512  0.339805   
                          soybean_large.csv           0.076039  0.152975   
neural-r:a16|nrmse        crx.csv                     0.321929  0.331049   
neural-r:mpg|nrmse        auto-mpg.csv                2.234662  3.403514   
neural-r:rings|nrmse      abalone.csv                 2.157315  2.600538   
neural-r:selector|nrmse   bupa.csv                    0.429799  0.442881   

                                                               
                                                          std  
algorithm                 dataset                              
gp-r:a16|nrmse            crx.csv                    0.000000  
gp-r:class|nrmse          hepatitis.csv              0.026263  
                          pima-indians-diabetes.csv  0.006394  
gp-r:diagnosis|nrmse      wcbreast_wdbc.csv          0.052918  
gp-r:mpg|nrmse            auto-mpg.csv               0.917775  
gp-r:outcome|nrmse        wcbreast_wpbc.csv          0.012323  
gp-r:rings|nrmse          abalone.csv                0.173115  
gp-r:selector|nrmse       bupa.csv                   0.014515  
neural-c:area|nrmse       forestfires.csv            0.000966  
neural-c:class|nrmse      hepatitis.csv              0.017165  
                          pima-indians-diabetes.csv  0.027356  
                          wine.csv                   0.059909  
neural-c:cover_type|nrmse covtype.csv                0.022559  
neural-c:crim|nrmse       housing.csv                0.000055  
neural-c:diagnosis|nrmse  wcbreast_wdbc.csv          0.013068  
neural-c:num|nrmse        heart_cleveland.csv        0.019778  
neural-c:outcome|nrmse    horse-colic.csv            0.010901  
                          wcbreast_wpbc.csv          0.034296  
neural-c:species|nrmse    iris.csv                   0.160781  
neural-c:type|nrmse       glass.csv                  0.015664  
                          soybean_large.csv          0.060550  
neural-r:a16|nrmse        crx.csv                    0.008736  
neural-r:mpg|nrmse        auto-mpg.csv               2.326207  
neural-r:rings|nrmse      abalone.csv                0.553925  
neural-r:selector|nrmse   bupa.csv                   0.011279

In [88]:
df[ (df.dataset == 'auto-mpg.csv') & (df.name=='experiment-2') & (df.algorithm.str.contains('neural'))]

,name,status,dataset,algorithm,predictors,cycle,result,result-raw,i1,i2,iterations,elapsed,info
120,experiment-2,done-macbook-pro,auto-mpg.csv,neural-r:mpg|nrmse,NaN,1,0.078099,2.440546,0,0,547,0,NaN
121,experiment-2,done-macbook-pro,auto-mpg.csv,neural-r:mpg|nrmse,NaN,2,0.077927,2.504313,0,0,595,0,NaN
122,experiment-2,done-macbook-pro,auto-mpg.csv,neural-r:mpg|nrmse,NaN,3,0.071180,2.234662,0,0,673,0,NaN
123,experiment-2,done-macbook-pro,auto-mpg.csv,neural-r:mpg|nrmse,NaN,4,0.073341,2.278064,0,0,547,0,NaN
124,experiment-2,done-macbook-pro,auto-mpg.csv,neural-r:mpg|nrmse,NaN,5,0.245422,7.559983,0,0,4249,2,NaN
